In [ ]:
import numpy as np
import pandas as pd
import talib
from glob import glob

np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))
from data_provider import read_data_path

market = 'ONT-BTC'
tf = '3m'

data_path = read_data_path() +'/BTC/' +market + '/' + tf
files = glob(data_path +'/*csv')
files

In [ ]:
def ma_crossover_ichimoku_01 (ticks, lead_window, lag_window):
    ma_lead = talib.MA(ticks, lead_window)
    ma_lag = talib.MA(ticks, lag_window)
    
    start = lag_window
    signals = np.zeros((len(ticks)))
    for cur_tick in range(start,len(ticks)):
        if ma_lead[cur_tick] > ma_lag[cur_tick] and ma_lag[cur_tick] > ma_lag[cur_tick-1]:
            signals[cur_tick] = 1
        elif ma_lead[cur_tick] < ma_lag[cur_tick] and ma_lag[cur_tick] < ma_lag[cur_tick-1]:
            signals[cur_tick] = -1
     
    #robot_log = np.transpose( np.array([ticks, ma_lead, ma_lag,  signals]))
    #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
    #return robot_log
    return signals

In [196]:
def ma_rsi_01 (prices, ma_window, rsi_window, sell_indicator):
    rsi = talib.RSI(prices * 100, rsi_window)
    ma = talib.MA(prices, ma_window)
    
    start = max(rsi_window, ma_window)
    signals = np.zeros((len(prices)))
    market_position = 0
    for cur_price in range(start,len(prices)):
        if prices[cur_price] > ma[cur_price] and market_position == 0:
            signals[cur_price] = 1
            market_position = 1
        elif  rsi[cur_price] > sell_indicator and market_position == 1:
            signals[cur_price] = -1
            market_position = 0
     
    #robot_log = np.transpose( np.array([prices, ma, rsi,  signals]))
    #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
    #return robot_log
    return signals

In [194]:
df = pd.read_csv(files[-1])
ticks = df['C'].as_matrix()
lead_window = 2
lag_window = 75


ss = ma_crossover_ichimoku_01(ticks, lead_window, lag_window)
ss[ss[:,3] !=0];

In [200]:
df = pd.read_csv(files[-1])
ticks = df['C'].as_matrix()
ma_window = 20
rsi_window = 14
sell_indicator = 75

ss = rsi_ma_01(ticks, ma_window, rsi_window, sell_indicator)
ss[ss[:,3] !=0];

In [225]:
scap = 0.1
cap = scap
cums = np.zeros(ss.shape[0])
j = 0
for i in ss:
    if i[3] == 1:
        ns = cap/i[0]
    elif i[3] == -1:
        cap = ns * i[0] 
    cums[j] = cap
    j = j +1

print(ss[-1,0]/ ss[0,0] * scap)
print(cap)
signals = -ss[:,3]
signals[-1] = 1
xx = np.cumprod(np.power(ss[:,0],signals)) * scap
print(xx)
ss[:,3]

0.18435895490300228
0.16553631588329534
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 126 126 126 126 126 126 126 126 ... 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 113 0.165]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [215]:
ss[:,3]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:
# signals 0-do nothing, 1-buy, -1-sell

def ma_01(ticks, ma_window):
    ma = talib.MA(ticks, ma_window)
    start = ma_window
    
    robot_log = np.zeros((len(ticks), 3))
    signals = []
    for cur_tick in range(start,len(ticks)):
        price = ticks[cur_tick]
        indicator = ma[cur_tick]
        if market_position == 0 and price > indicator:
            signal = 1
        elif market_position == 1 and price < indicator and bars_since_entry > 0:
            signal = -1
        else:
            signal = 0
            
        bars_since_entry = bars_since_entry + 1
        signals.append(signal)
        #robot_log[cur_tick] = [price, indicator, signal]
    
    
    #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
    return signals

In [ ]:
 
def rsi_01 (ticks, window, buy_indicator, sell_indicator):
    indicators_array = talib.RSI(ticks * 100, window)
    start = window
    
    robot_log = np.zeros((len(ticks), 3))
    signals = []
    for cur_tick in range(start,len(ticks)):
        price = ticks[cur_tick]
        indicator = indicators_array[cur_tick]
        if indicator < buy_indicator:
            signal = 1
        elif indicator > sell_indicator:
            signal = -1
        else:
            signal = 0
            
        signals.append(signal)
        #robot_log[cur_tick] = [price, indicator,  signal]
    
    
    #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
    return signals

In [1]:
def ma_crossover_01(ticks, lead_window, lag_window, threshold = 0.025):
    arr = np.zeros((len(ticks), 3))
    arr[:, 0] = ticks
    arr[:, 1] = talib.MA(ticks, lead_window)
    arr[:, 2] = talib.MA(ticks, lag_window)
    ma_df = pd.DataFrame(arr, columns = ['close', 'lead', 'lag'])
    
    ma_df['lead-lag'] = ma_df['lead'] - ma_df['lag']
    ma_df['pc_diff'] = ma_df['lead-lag'] / ma_df['close']
    ma_df['signals'] = np.where(ma_df['pc_diff'] > threshold, 1, 0)
    ma_df['signals'] = np.where(ma_df['pc_diff'] < -threshold, -1, ma_df['signals'])
    
    #ma_df.to_csv('Logs/test.csv', robot_log, delimiter=',')
    
    return ma_df['signals']    

In [7]:
def ma_crossover_02(ticks, lead_window, lag_window, threshold = 0.025):
    #Fastest
    return 'Not tested'
    arr = np.zeros((len(ticks), 6))
    arr[:, 0] = ticks
    arr[:, 1] = talib.MA(ticks, lead_window)
    arr[:, 2] = talib.MA(ticks, lag_window)
    arr[:lag_window + 1, 1] = 1
    arr[:lag_window + 1, 2] = 1
        
    
    arr[:, 3] = arr[:,1] - arr[:,2]
    arr[:, 4] = arr[:,3] / arr[:,0]
    arr[:, 5] = np.where(arr[:, 4] > threshold, 1, 0)
    arr[:, 5] = np.where(arr[:, 4] < -threshold, -1, arr[:, 5])
    
    return arr[:, 5]   

In [9]:
def ma_crossover_03(ticks, lead_window, lag_window, threshold = 0.025):
    #Trade-off between speed and readability
    return 'Not tested'
    arr = np.zeros((len(ticks), 6))
    dtype = [('close', float), ('lead', float), ('lag', float),('lead-lag', float), ('pc_diff', float), ('signals', float)] 
    arr.dtype = dtype
    
    ticks_shape = (len(ticks),1)
    arr['close'] = np.resize(ticks, ticks_shape)
    arr['lead'] = np.resize(talib.MA(ticks, lead_window), ticks_shape)
    arr['lag'] = np.resize(talib.MA(ticks, lag_window), ticks_shape)
    arr['lead'] = 1
    arr['lag'] = 1
        
    
    arr['lead-lag'] = arr['lead'] - arr['lag']
    arr['pc_diff'] = arr['lead-lag'] / arr['close']
    arr['signals'] = np.where(arr['pc_diff'] > threshold, 1, 0)
    arr['signals'] = np.where(arr['pc_diff'] < -threshold, -1, arr['signals'])
    
    return arr['signals']   

0 3
1 5
2 7
